# Initalisation

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [70]:
## https://pytorch.org/tutorials/beginner/chatbot_tutorial.html

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math

## ENV VARs:
Algorithm="LSTM" #User defined algorithm: set to RNN/LSTM/MLP/DNN

#from google.colab import drive
#drive.mount('/content/drive')

USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

corpus_name = "cornell movie-dialogs corpus"
corpus_folder = os.path.join("data", corpus_name)
corpus = os.path.join("/content/drive/My Drive/", corpus_folder)
def printLines(file, n=10):
    with open(file, 'rb') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)

printLines(os.path.join(corpus, "movie_lines.txt"))

b'L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!\n'
b'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!\n'
b'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.\n'
b'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?\n'
b"L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.\n"
b'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow\n'
b"L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.\n"
b'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No\n'
b'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?\n'
b'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?\n'


# Cornell Movie Dialog Dataset

In [19]:
#from google.colab import drive
#drive.mount('/content/drive')
Dataset = 1
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

corpus_name = "cornell movie-dialogs corpus"
corpus_folder = os.path.join("data", corpus_name)
corpus = os.path.join("/content/drive/My Drive/", corpus_folder)
def printLines(file, n=10):
    with open(file, 'rb') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)

printLines(os.path.join(corpus, "movie_lines.txt"))

b'L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!\n'
b'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!\n'
b'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.\n'
b'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?\n'
b"L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.\n"
b'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow\n'
b"L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.\n"
b'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No\n'
b'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?\n'
b'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?\n'


# SQuAD 2.0 Dataset

Loading the SQuAD 2.0 (The Stanford Question Answering Dataset), taken from: 

https://www.kaggle.com/stanfordu/stanford-question-answering-dataset

In [49]:
Dataset = 2

import json
import pandas as pd
import re
from keras.preprocessing.text import text_to_word_sequence
corpus_name = "SQUAD"
corpus_folder = os.path.join("data", corpus_name)
corpus = os.path.join("/content/drive/My Drive/", corpus_folder)
json_data = os.path.join(corpus, "train-v2.0.json")
train_df = pd.read_json(json_data)

with open(json_data, 'r') as file:
  data = json.loads(file.read())

# Define path to new file
datafile = os.path.join(corpus, "formatted_squad_lines.txt") #will be written to below as a csv

delimiter = '\t' 
# Unescape the delimiter -- this adds a '\t' to the .csv file in place of an escape
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

# Define path to new file
datafile = os.path.join(corpus, "formatted_movie_lines.txt") #will be written to below as a csv

with open(datafile, 'w', encoding='utf-8') as outputfile:
  writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
  for i in range(train_df['data'].shape[0]):
      topic = train_df['data'].iloc[i]['paragraphs']
      for sub_para in topic:
        for q_a in sub_para['qas']:
            if(not(q_a['is_impossible'])):                    
                question = q_a['question']
                answer = q_a['answers'][0]['text']
                inputLine = re.sub("[^a-zA-Z ]+", "", question)
                targetLine = re.sub("[^a-zA-Z ]+", "", answer)
                if inputLine and targetLine:
                  qa_pairs = []
                  qa_pairs.append(inputLine)                          
                  qa_pairs.append(targetLine)
                  writer.writerow(qa_pairs)

printLines(datafile) 

b'When did Beyonce start becoming popular\tin the late s\n'
b'What areas did Beyonce compete in when she was growing up\tsinging and dancing\n'
b'In what city and state did Beyonce  grow up \tHouston Texas\n'
b'In which decade did Beyonce become famous\tlate s\n'
b'In what RB group was she the lead singer\tDestinys Child\n'
b'What album made her a worldwide known artist\tDangerously in Love\n'
b'Who managed the Destinys Child group\tMathew Knowles\n'
b'When did Beyonc rise to fame\tlate s\n'
b'What role did Beyonc have in Destinys Child\tlead singer\n'
b'What was the first album Beyonc released as a solo artist\tDangerously in Love\n'


# Loading
(Cornell Dataset only)

In [47]:
######################################################################
# Create formatted data file
# ~~~~~~~~~~~~~~~~~~~~~~~~~~
#
# For convenience, we'll create a nicely formatted data file in which each line
# contains a tab-separated *query sentence* and a *response sentence* pair.
#
# The following functions facilitate the parsing of the raw
# *movie_lines.txt* data file.
#
# -  ``loadLines`` splits each line of the file into a dictionary of
#    fields (lineID, characterID, movieID, character, text)
# -  ``loadConversations`` groups fields of lines from ``loadLines`` into
#    conversations based on *movie_conversations.txt*
# -  ``extractSentencePairs`` extracts pairs of sentences from
#    conversations
#
# Splits each line of the file into a dictionary of fields
def loadLines(fileName, fields):
    lines = {}
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(" +++$+++ ")
            # Extract fields
            lineObj = {}
            for i, field in enumerate(fields):
                lineObj[field] = values[i]
            lines[lineObj['lineID']] = lineObj
    return lines


# Groups fields of lines from `loadLines` into conversations based on *movie_conversations.txt*
def loadConversations(fileName, lines, fields):
    conversations = []
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(" +++$+++ ")
            # Extract fields
            convObj = {}
            for i, field in enumerate(fields):
                convObj[field] = values[i]
            # Convert string to list (convObj["utteranceIDs"] == "['L598485', 'L598486', ...]")
            utterance_id_pattern = re.compile('L[0-9]+')
            lineIds = utterance_id_pattern.findall(convObj["utteranceIDs"])
            # Reassemble lines
            convObj["lines"] = []
            for lineId in lineIds:
                convObj["lines"].append(lines[lineId])
            conversations.append(convObj)
    return conversations


# Extracts pairs of sentences from conversations
def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations:
        # Iterate over all the lines of the conversation
        for i in range(len(conversation["lines"]) - 1):  # We ignore the last line (no answer for it)
            inputLine = conversation["lines"][i]["text"].strip()
            targetLine = conversation["lines"][i+1]["text"].strip()
            # Filter wrong samples (if one of the lists is empty)
            if inputLine and targetLine:
                qa_pairs.append([inputLine, targetLine])
    return qa_pairs

# Create Dataset
(Cornell Dataset only)

In [48]:
######################################################################
# Now we’ll call these functions and create the file. We’ll call it
# *formatted_movie_lines.txt*.
#

# Define path to new file
datafile = os.path.join(corpus, "formatted_movie_lines.txt") #will be written to below as a csv

delimiter = '\t' 
# Unescape the delimiter -- this adds a '\t' to the .csv file in place of an escape
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

# Initialize lines dict, conversations list, and field ids
lines = {}
conversations = []
MOVIE_LINES_FIELDS = ["lineID", "characterID", "movieID", "character", "text"]
MOVIE_CONVERSATIONS_FIELDS = ["character1ID", "character2ID", "movieID", "utteranceIDs"]

# Load lines and process conversations
print("\nProcessing corpus...")
lines = loadLines(os.path.join(corpus, "movie_lines.txt"), MOVIE_LINES_FIELDS)
print("\nLoading conversations...")
conversations = loadConversations(os.path.join(corpus, "movie_conversations.txt"),
                                  lines, MOVIE_CONVERSATIONS_FIELDS)

# Write new csv file
print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in extractSentencePairs(conversations):
        writer.writerow(pair)

# Print a sample of lines
print("\nSample lines from file:")
printLines(datafile)


Processing corpus...


FileNotFoundError: ignored

# Load and Trim Data

In [50]:
######################################################################
# Load and trim data
# ~~~~~~~~~~~~~~~~~~
#
# Our next order of business is to create a vocabulary and load
# query/response sentence pairs into memory.
#
# Note that we are dealing with sequences of **words**, which do not have
# an implicit mapping to a discrete numerical space. Thus, we must create
# one by mapping each unique word that we encounter in our dataset to an
# index value.
#
# For this we define a ``Voc`` class, which keeps a mapping from words to
# indexes, a reverse mapping of indexes to words, a count of each word and
# a total word count. The class provides methods for adding a word to the
# vocabulary (``addWord``), adding all words in a sentence
# (``addSentence``) and trimming infrequently seen words (``trim``). More
# on trimming later.
#

# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

# Create Voc and Pairs

In [51]:
######################################################################
# Now we can assemble our vocabulary and query/response sentence pairs.
# Before we are ready to use this data, we must perform some
# preprocessing.
#
# First, we must convert the Unicode strings to ASCII using
# ``unicodeToAscii``. Next, we should convert all letters to lowercase and
# trim all non-letter characters except for basic punctuation
# (``normalizeString``). Finally, to aid in training convergence, we will
# filter out sentences with length greater than the ``MAX_LENGTH``
# threshold (``filterPairs``).
#

MAX_LENGTH = 30  # Maximum sentence length to consider

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# Read query/response pairs and return a voc object
def readVocs(datafile, corpus_name):
    print("Reading lines...")
    # Read the file and split into lines
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

# Returns True iff both sentences in a pair 'p' are under the MAX_LENGTH threshold
def filterPair(p):
    # Input sequences need to preserve the last word for EOS token
    if len(p)>1:      
      return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH
    return 0  

# Filter pairs using filterPair condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

# Using the functions defined above, return a populated voc object and pairs list
def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs


# Load/Assemble voc and pairs
save_dir = os.path.join("data", "save")

voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)
# Print some pairs to validate
print("\npairs:")
for pair in pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 77431 sentence pairs
Trimmed to 77396 sentence pairs
Counting words...
Counted words: 53643

pairs:
['when did beyonce start becoming popular', 'in the late s']
['what areas did beyonce compete in when she was growing up', 'singing and dancing']
['in what city and state did beyonce grow up', 'houston texas']
['in which decade did beyonce become famous', 'late s']
['in what rb group was she the lead singer', 'destinys child']
['what album made her a worldwide known artist', 'dangerously in love']
['who managed the destinys child group', 'mathew knowles']
['when did beyonc rise to fame', 'late s']
['what role did beyonc have in destinys child', 'lead singer']
['what was the first album beyonc released as a solo artist', 'dangerously in love']


# Trim Data

In [52]:

######################################################################
# Another tactic that is beneficial to achieving faster convergence during
# training is trimming rarely used words out of our vocabulary. Decreasing
# the feature space will also soften the difficulty of the function that
# the model must learn to approximate. We will do this as a two-step
# process:
#
# 1) Trim words used under ``MIN_COUNT`` threshold using the ``voc.trim``
#    function.
#
# 2) Filter out pairs with trimmed words.
#

MIN_COUNT = 3    # Minimum word count threshold for trimming

def trimRareWords(voc, pairs, MIN_COUNT):
    # Trim words used under the MIN_COUNT from the voc
    voc.trim(MIN_COUNT)
    # Filter out pairs with trimmed words
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # Check input sentence
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        # Check output sentence
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        # Only keep pairs that do not contain trimmed word(s) in their input or output sentence
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


# Trim voc and pairs
pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 20770 / 53640 = 0.3872
Trimmed from 77396 pairs to 47678, 0.6160 of total


# Prepare Data For Models

In [53]:
######################################################################
# Prepare Data for Models
# -----------------------
#
# Although we have put a great deal of effort into preparing and massaging our
# data into a nice vocabulary object and list of sentence pairs, our models
# will ultimately expect numerical torch tensors as inputs. One way to
# prepare the processed data for the models can be found in the `seq2seq
# translation
# tutorial <https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html>`__.
# In that tutorial, we use a batch size of 1, meaning that all we have to
# do is convert the words in our sentence pairs to their corresponding
# indexes from the vocabulary and feed this to the models.
#
# However, if you’re interested in speeding up training and/or would like
# to leverage GPU parallelization capabilities, you will need to train
# with mini-batches.
#
# Using mini-batches also means that we must be mindful of the variation
# of sentence length in our batches. To accomodate sentences of different
# sizes in the same batch, we will make our batched input tensor of shape
# *(max_length, batch_size)*, where sentences shorter than the
# *max_length* are zero padded after an *EOS_token*.
#
# If we simply convert our English sentences to tensors by converting
# words to their indexes(\ ``indexesFromSentence``) and zero-pad, our
# tensor would have shape *(batch_size, max_length)* and indexing the
# first dimension would return a full sequence across all time-steps.
# However, we need to be able to index our batch along time, and across
# all sequences in the batch. Therefore, we transpose our input batch
# shape to *(max_length, batch_size)*, so that indexing across the first
# dimension returns a time step across all sentences in the batch. We
# handle this transpose implicitly in the ``zeroPadding`` function.
#
# .. figure:: /_static/img/chatbot/seq2seq_batches.png
#    :align: center
#    :alt: batches
#
# The ``inputVar`` function handles the process of converting sentences to
# tensor, ultimately creating a correctly shaped zero-padded tensor. It
# also returns a tensor of ``lengths`` for each of the sequences in the
# batch which will be passed to our decoder later.
#
# The ``outputVar`` function performs a similar function to ``inputVar``,
# but instead of returning a ``lengths`` tensor, it returns a binary mask
# tensor and a maximum target sentence length. The binary mask tensor has
# the same shape as the output target tensor, but every element that is a
# *PAD_token* is 0 and all others are 1.
#
# ``batch2TrainData`` simply takes a bunch of pairs and returns the input
# and target tensors using the aforementioned functions.
#

def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    #return list(itertools.zip_longest(l, fillvalue=fillvalue))
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    #### JMc: flip back to (batch_size, max_length) for CNN rather than time series data
    #if Algorithm is 'CNN':
      #input_batch = torch.LongTensor(padList).transpose(0, 1)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    #### JMc: flip back to (batch_size, max_length) for CNN rather than time series data
    #if Algorithm is 'CNN':
      #input_batch = torch.LongTensor(padList).transpose(0, 1)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)

    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


# Example for validation
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

#print("input_variable:", input_variable)
#print("lengths:", lengths)
#print("target_variable:", target_variable)
#print("mask:", mask)
#print("max_target_len:", max_target_len)


# Encoders 

In [71]:
######################################################################
# Define Models
# -------------
#
# Seq2Seq Model
# ~~~~~~~~~~~~~
#
# The brains of our chatbot is a sequence-to-sequence (seq2seq) model. The
# goal of a seq2seq model is to take a variable-length sequence as an
# input, and return a variable-length sequence as an output using a
# fixed-sized model.
#
# `Sutskever et al. <https://arxiv.org/abs/1409.3215>`__ discovered that
# by using two separate recurrent neural nets together, we can accomplish
# this task. One RNN acts as an **encoder**, which encodes a variable
# length input sequence to a fixed-length context vector. In theory, this
# context vector (the final hidden layer of the RNN) will contain semantic
# information about the query sentence that is input to the bot. The
# second RNN is a **decoder**, which takes an input word and the context
# vector, and returns a guess for the next word in the sequence and a
# hidden state to use in the next iteration.
#
# .. figure:: /_static/img/chatbot/seq2seq_ts.png
#    :align: center
#    :alt: model
#
# Image source:
# https://jeddy92.github.io/JEddy92.github.io/ts_seq2seq_intro/
#


######################################################################
# Encoder
# ~~~~~~~
#
# The encoder RNN iterates through the input sentence one token
# (e.g. word) at a time, at each time step outputting an “output” vector
# and a “hidden state” vector. The hidden state vector is then passed to
# the next time step, while the output vector is recorded. The encoder
# transforms the context it saw at each point in the sequence into a set
# of points in a high-dimensional space, which the decoder will use to
# generate a meaningful output for the given task.
#
# At the heart of our encoder is a multi-layered Gated Recurrent Unit,
# invented by `Cho et al. <https://arxiv.org/pdf/1406.1078v3.pdf>`__ in
# 2014. We will use a bidirectional variant of the GRU, meaning that there
# are essentially two independent RNNs: one that is fed the input sequence
# in normal sequential order, and one that is fed the input sequence in
# reverse order. The outputs of each network are summed at each time step.
# Using a bidirectional GRU will give us the advantage of encoding both
# past and future context.
#
# Bidirectional RNN:
#
# .. figure:: /_static/img/chatbot/RNN-bidirectional.png
#    :width: 70%
#    :align: center
#    :alt: rnn_bidir
#
# Image source: https://colah.github.io/posts/2015-09-NN-Types-FP/
#
# Note that an ``embedding`` layer is used to encode our word indices in
# an arbitrarily sized feature space. For our models, this layer will map
# each word to a feature space of size *hidden_size*. When trained, these
# values should encode semantic similarity between similar meaning words.
#
# Finally, if passing a padded batch of sequences to an RNN module, we
# must pack and unpack padding around the RNN pass using
# ``nn.utils.rnn.pack_padded_sequence`` and
# ``nn.utils.rnn.pad_packed_sequence`` respectively.
#
# **Computation Graph:**
#
#    1) Convert word indexes to embeddings.
#    2) Pack padded batch of sequences for RNN module.
#    3) Forward pass through GRU.
#    4) Unpack padding.
#    5) Sum bidirectional GRU outputs.
#    6) Return output and final hidden state.
#
# **Inputs:**
#
# -  ``input_seq``: batch of input sentences; shape=\ *(max_length,
#    batch_size)*
# -  ``input_lengths``: list of sentence lengths corresponding to each
#    sentence in the batch; shape=\ *(batch_size)*
# -  ``hidden``: hidden state; shape=\ *(n_layers x num_directions,
#    batch_size, hidden_size)*
#
# **Outputs:**
#
# -  ``outputs``: output features from the last hidden layer of the GRU
#    (sum of bidirectional outputs); shape=\ *(max_length, batch_size,
#    hidden_size)*
# -  ``hidden``: updated hidden state from GRU; shape=\ *(n_layers x
#    num_directions, batch_size, hidden_size)*
#
#

class Encoder(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        if Algorithm is 'RNN':
          self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                            dropout=(0 if n_layers == 1 else dropout), bidirectional=True)
          
        if Algorithm is 'LSTM':
          self.LSTM = nn.LSTM(hidden_size, hidden_size, n_layers,
                            dropout=(0 if n_layers == 1 else dropout))  ## No bidirectional for LSTM (at the  min)       
          #self.gru = nn.LSTM(hidden_size, hidden_size, n_layers, bidirectional=True)      
        if Algorithm is 'MLP':
            self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                              dropout=(0 if n_layers == 1 else dropout), bidirectional=True)
            self.fc1 = nn.Linear(hidden_size, hidden_size)
            self.fc2 = nn.Linear(hidden_size, hidden_size)
            #self.layers = nn.Sequential(
              #nn.Linear(hidden_size, int(hidden_size/2)),
              #nn.ReLU(),
              #F.dropout(intermediate, p=0.5),
              #nn.Linear(int(hidden_size/2), hidden_size)              
            #)   
        if Algorithm is 'DNN':
            self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                              dropout=(0 if n_layers == 1 else dropout), bidirectional=True)
            self.fc1 = nn.Linear(hidden_size, hidden_size)
            self.fc2 = nn.Linear(hidden_size, hidden_size)    
            self.fc3 = nn.Linear(hidden_size, hidden_size)
            self.fc4 = nn.Linear(hidden_size, hidden_size)    
            self.fc5 = nn.Linear(hidden_size, hidden_size)
            self.fc6 = nn.Linear(hidden_size, hidden_size)    
        #if Algorithm is 'CNN':  
          #self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                              #dropout=(0 if n_layers == 1 else dropout), bidirectional=True)
          # need to add padding to Conv1d to prevent inputs from last column being lost, see: https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html 
          #self.conv1 = nn.Conv1d(in_channels=64, out_channels=10, kernel_size=3, padding=1) 
          #self.conv2 = nn.Conv1d(in_channels=10, out_channels=64, kernel_size=3, padding=1)



    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        
        #print("Train - pass through Encoder")
        # Forward pass through GRU
        if Algorithm is 'RNN':
          # Pack padded batch of sequences for RNN module
          packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
          outputs, hidden = self.gru(packed, hidden)
        if Algorithm is 'LSTM':
          # Pack padded batch of sequences for RNN module
          packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
          outputs, hidden = self.LSTM(packed, hidden)
        if Algorithm is 'DNN':
          intermediate1 = F.relu(self.fc1(embedded))
          intermediate2 = F.relu(self.fc2(intermediate1))
          intermediate3 = F.relu(self.fc3(intermediate2))
          intermediate4 = F.relu(self.fc4(intermediate3))
          intermediate5 = F.relu(self.fc5(intermediate4))
          embeddedMLP = self.fc6(F.dropout(intermediate5, p=0.5))
          #embeddedMLP = self.layers(embedded) 
          # Pack padded batch of sequences for RNN module
          #print("embeddedMLP size: ", embeddedMLP.size())
          packed = nn.utils.rnn.pack_padded_sequence(embeddedMLP, input_lengths)
          outputs, hidden = self.gru(packed, hidden) 
        if Algorithm is 'MLP':
          intermediate = F.relu(self.fc1(embedded))
          embeddedMLP = self.fc2(F.dropout(intermediate, p=0.5))
          #embeddedMLP = self.layers(embedded) 
          # Pack padded batch of sequences for RNN module
          print("embeddedMLP size: ", embeddedMLP.size())
          packed = nn.utils.rnn.pack_padded_sequence(embeddedMLP, input_lengths)
          outputs, hidden = self.gru(packed, hidden) 
        #if Algorithm is 'CNN':
          #intermediate = self.conv1(embedded)
          #embeddedCNN = self.conv2(intermediate) 
          #print("embeddedCNN size: ", embeddedCNN.size())
          #embeddedCNN = embeddedCNN.squeeze()
          #print("embeddedCNN size after squeeze: ", embeddedCNN.size())
          #packed = nn.utils.rnn.pack_padded_sequence(embeddedCNN, input_lengths)
          #outputs, hidden = self.gru(packed, hidden) 
        #print("Train - pass through Encoder has finished")
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        #print("self.hidden_size")
        #print(self.hidden_size)
        #print("encoder hidden")
        #print(len(hidden))
        if Algorithm is not 'LSTM': ################# remove the bidirectional summing for LSTM (for now) ###################
          outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden


# Decoders

In [29]:
######################################################################
# Decoder
# ~~~~~~~
#
# The decoder RNN generates the response sentence in a token-by-token
# fashion. It uses the encoder’s context vectors, and internal hidden
# states to generate the next word in the sequence. It continues
# generating words until it outputs an *EOS_token*, representing the end
# of the sentence. A common problem with a vanilla seq2seq decoder is that
# if we rely soley on the context vector to encode the entire input
# sequence’s meaning, it is likely that we will have information loss.
# This is especially the case when dealing with long input sequences,
# greatly limiting the capability of our decoder.
#
# To combat this, `Bahdanau et al. <https://arxiv.org/abs/1409.0473>`__
# created an “attention mechanism” that allows the decoder to pay
# attention to certain parts of the input sequence, rather than using the
# entire fixed context at every step.
#
# At a high level, attention is calculated using the decoder’s current
# hidden state and the encoder’s outputs. The output attention weights
# have the same shape as the input sequence, allowing us to multiply them
# by the encoder outputs, giving us a weighted sum which indicates the
# parts of encoder output to pay attention to. `Sean
# Robertson’s <https://github.com/spro>`__ figure describes this very
# well:
#
# .. figure:: /_static/img/chatbot/attn2.png
#    :align: center
#    :alt: attn2
#
# `Luong et al. <https://arxiv.org/abs/1508.04025>`__ improved upon
# Bahdanau et al.’s groundwork by creating “Global attention”. The key
# difference is that with “Global attention”, we consider all of the
# encoder’s hidden states, as opposed to Bahdanau et al.’s “Local
# attention”, which only considers the encoder’s hidden state from the
# current time step. Another difference is that with “Global attention”,
# we calculate attention weights, or energies, using the hidden state of
# the decoder from the current time step only. Bahdanau et al.’s attention
# calculation requires knowledge of the decoder’s state from the previous
# time step. Also, Luong et al. provides various methods to calculate the
# attention energies between the encoder output and decoder output which
# are called “score functions”:
#
# .. figure:: /_static/img/chatbot/scores.png
#    :width: 60%
#    :align: center
#    :alt: scores
#
# where :math:`h_t` = current target decoder state and :math:`\bar{h}_s` =
# all encoder states.
#
# Overall, the Global attention mechanism can be summarized by the
# following figure. Note that we will implement the “Attention Layer” as a
# separate ``nn.Module`` called ``Attn``. The output of this module is a
# softmax normalized weights tensor of shape *(batch_size, 1,
# max_length)*.
#
# .. figure:: /_static/img/chatbot/global_attn.png
#    :align: center
#    :width: 60%
#    :alt: global_attn
#

# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        #if Algorithm is not 'CNN':
          #attn_energies = attn_energies.t()
        attn_energies = attn_energies.t()
        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)


# Luong Attn Decoder RNN

In [13]:
######################################################################
# Now that we have defined our attention submodule, we can implement the
# actual decoder model. For the decoder, we will manually feed our batch
# one time step at a time. This means that our embedded word tensor and
# GRU output will both have shape *(1, batch_size, hidden_size)*.
#
# **Computation Graph:**
#
#    1) Get embedding of current input word.
#    2) Forward through unidirectional GRU.
#    3) Calculate attention weights from the current GRU output from (2).
#    4) Multiply attention weights to encoder outputs to get new "weighted sum" context vector.
#    5) Concatenate weighted context vector and GRU output using Luong eq. 5.
#    6) Predict next word using Luong eq. 6 (without softmax).
#    7) Return output and final hidden state.
#
# **Inputs:**
#
# -  ``input_step``: one time step (one word) of input sequence batch;
#    shape=\ *(1, batch_size)*
# -  ``last_hidden``: final hidden layer of GRU; shape=\ *(n_layers x
#    num_directions, batch_size, hidden_size)*
# -  ``encoder_outputs``: encoder model’s output; shape=\ *(max_length,
#    batch_size, hidden_size)*
#
# **Outputs:**
#
# -  ``output``: softmax normalized tensor giving probabilities of each
#    word being the correct next word in the decoded sequence;
#    shape=\ *(batch_size, voc.num_words)*
# -  ``hidden``: final hidden state of GRU; shape=\ *(n_layers x
#    num_directions, batch_size, hidden_size)*
#

class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        if Algorithm is not 'LSTM':
          self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
         
        else:
          #self.gru = nn.LSTM(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
          #print("hidden_size:")
          #print(hidden_size)
          lstm_hidden_tensor = torch.empty(n_layers*2,batch_size, hidden_size)
          #lstm_hidden_tensor = torch.Tensor()
          #lstm_hidden_tensor = (2, 64, 1000)  #### needs to be  (num_layers * num_directions, batch, hidden_size)
          self.LSTM = nn.LSTM(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))           

        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        #print("embedded")
        #print(embedded.size())

        # Forward through unidirectional GRU
        if Algorithm is not 'LSTM':
          rnn_output, hidden = self.gru(embedded, last_hidden)
        else: 
          rnn_output, hidden = self.LSTM(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        #if Algorithm is not 'CNN':
          #context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        #else: 
          #context = attn_weights.bmm(encoder_outputs)
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))  
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        #print("rnn_output")
        #print(rnn_output.size())

        rnn_output = rnn_output.squeeze(0)

        #print("rnn_output after squeeze")
        #print(rnn_output.size())
        #print("context")
        #print(context.size())

        context = context.squeeze(1)

        #print("context after squeeze")
        #print(context.size())

        concat_input = torch.cat((rnn_output, context), 1)

        #print("context")        
        #print(context.size())

        concat_output = torch.tanh(self.concat(concat_input))
       
        #print("concat_output")    
        #print(concat_output.size())

        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
         
        #print("output after self.out()")    
        #print(output.size())

        output = F.softmax(output, dim=1)
        
        #print("output after softmax()")    
        #print(output.size())
        # Return output and final hidden state
        return output, hidden


# Masked loss

In [14]:
######################################################################
# Define Training Procedure
# -------------------------
#
# Masked loss
# ~~~~~~~~~~~
#
# Since we are dealing with batches of padded sequences, we cannot simply
# consider all elements of the tensor when calculating loss. We define
# ``maskNLLLoss`` to calculate our loss based on our decoder’s output
# tensor, the target tensor, and a binary mask tensor describing the
# padding of the target tensor. This loss function calculates the average
# negative log likelihood of the elements that correspond to a *1* in the
# mask tensor.
#

def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

# Train

In [72]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    # Lengths for rnn packing should always be on the cpu
    lengths = lengths.to("cpu")

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder        
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

# Training Iterations

In [73]:
######################################################################
# Training iterations
# ~~~~~~~~~~~~~~~~~~~
#
# It is finally time to tie the full training procedure together with the
# data. The ``trainIters`` function is responsible for running
# ``n_iterations`` of training given the passed models, optimizers, data,
# etc. This function is quite self explanatory, as we have done the heavy
# lifting with the ``train`` function.
#
# One thing to note is that when we save our model, we save a tarball
# containing the encoder and decoder state_dicts (parameters), the
# optimizers’ state_dicts, the loss, the iteration, etc. Saving the model
# in this way will give us the ultimate flexibility with the checkpoint.
# After loading a checkpoint, we will be able to use the model parameters
# to run inference, or we can continue training right where we left off.
#

def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

################################################################## CREAT GRAPH HERE #######################################################    
    import matplotlib.pyplot as plt
    import numpy as np
    import pickle
    plt.figure()
    x=[]
    y=[]

    
####################################################

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch


        loss= train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

################### Graph Iteration ############################
        # Run a training iteration with batch
        x.append(iteration)
        y.append(loss)
     
####################################################
        


        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

################################################################## ADD TO GRAPH HERE #######################################################        
    #plt.plot(x,y)
    #plt.show()
    #ax = plt.subplot(111)
    #x = np.exp(x)
    #y = np.linspace(0, 10)
    plt.plot(x, y)
    plt.show()

    #if Dataset is 1:
    #  if Algorithm is 'RNN':        
     #   with open(os.path.join(corpus, '1_RNNimg.pickle'),'wb') as fid:
      #    pickle.dump(ax, fid)
        #pickle.dump(ax, file('1_RNNimg.pickle', 'w'))
        #plt.savefig('/content/drive/My Drive/data/1_RNNimg.png')

      #if Algorithm is 'LSTM':
       # with open(os.path.join(corpus,'1_LSTMimg.pickle'),'wb') as fid:
        #  pickle.dump(ax, fid)
        #pickle.dump(ax, file('1_LSTMimg.pickle', 'w'))
        #plt.savefig('/content/drive/My Drive/data/1_LSTMimg.png')  

      #if Algorithm is 'MLP':
       # with open(os.path.join(corpus,'1_MLPimg.pickle'),'wb') as fid:
        #  pickle.dump(ax, fid)
        #plt.savefig('/content/drive/My Drive/data/1_MLPimg.png')  

      #if Algorithm is 'DNN':
       # with open(os.path.join(corpus,'1_DNNimg.pickle'),'wb') as fid:
        #  pickle.dump(ax, fid)
        #plt.savefig('/content/drive/My Drive/data/1_DNNimg.png')  

    #if Dataset is 2:      
      #if Algorithm is 'RNN':
       # with open(os.path.join(corpus,'2_RNNimg.pickle'),'wb') as fid:
        #  pickle.dump(ax, fid)
        #plt.savefig('/content/drive/My Drive/data/2_RNNimg.png')

      #if Algorithm is 'LSTM':
       # with open(os.path.join(corpus,'2_LSTMimg.pickle'),'wb') as fid:
        #  pickle.dump(ax, fid)
        #plt.savefig('/content/drive/My Drive/data/2_LSTMimg.png')  

      #if Algorithm is 'MLP':
       # with open(os.path.join(corpus, '2_MLPimg.pickle'),'wb') as fid:
        #  pickle.dump(ax, fid)
        #plt.savefig('/content/drive/My Drive/data/2_MLPimg.png')  

      #if Algorithm is 'DNN':
       # with open(os.path.join(corpus,'2_DNNimg.pickle'),'wb') as fid:
        #  pickle.dump(ax, fid)

        #plt.savefig('/content/drive/My Drive/data/2_DNNimg.png')              

######################

# Greedy Search Decoder

In [57]:
######################################################################
# Define Evaluation
# -----------------
#
# After training a model, we want to be able to talk to the bot ourselves.
# First, we must define how we want the model to decode the encoded input.
#
# Greedy decoding
# ~~~~~~~~~~~~~~~
#
# Greedy decoding is the decoding method that we use during training when
# we are **NOT** using teacher forcing. In other words, for each time
# step, we simply choose the word from ``decoder_output`` with the highest
# softmax value. This decoding method is optimal on a single time-step
# level.
#
# To facilite the greedy decoding operation, we define a
# ``GreedySearchDecoder`` class. When run, an object of this class takes
# an input sequence (``input_seq``) of shape *(input_seq length, 1)*, a
# scalar input length (``input_length``) tensor, and a ``max_length`` to
# bound the response sentence length. The input sentence is evaluated
# using the following computational graph:
#
# **Computation Graph:**
#
#    1) Forward input through encoder model.
#    2) Prepare encoder's final hidden layer to be first hidden input to the decoder.
#    3) Initialize decoder's first input as SOS_token.
#    4) Initialize tensors to append decoded words to.
#    5) Iteratively decode one word token at a time:
#        a) Forward pass through decoder.
#        b) Obtain most likely word token and its softmax score.
#        c) Record token and score.
#        d) Prepare current token to be next decoder input.
#    6) Return collections of word tokens and scores.
#

class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

# Evaluate

In [74]:
######################################################################
# Evaluate my text
# ~~~~~~~~~~~~~~~~
#
# Now that we have our decoding method defined, we can write functions for
# evaluating a string input sentence. The ``evaluate`` function manages
# the low-level process of handling the input sentence. We first format
# the sentence as an input batch of word indexes with *batch_size==1*. We
# do this by converting the words of the sentence to their corresponding
# indexes, and transposing the dimensions to prepare the tensor for our
# models. We also create a ``lengths`` tensor which contains the length of
# our input sentence. In this case, ``lengths`` is scalar because we are
# only evaluating one sentence at a time (batch_size==1). Next, we obtain
# the decoded response sentence tensor using our ``GreedySearchDecoder``
# object (``searcher``). Finally, we convert the response’s indexes to
# words and return the list of decoded words.
#
# ``evaluateInput`` acts as the user interface for our chatbot. When
# called, an input text field will spawn in which we can enter our query
# sentence. After typing our input sentence and pressing *Enter*, our text
# is normalized in the same way as our training data, and is ultimately
# fed to the ``evaluate`` function to obtain a decoded output sentence. We
# loop this process, so we can keep chatting with our bot until we enter
# either “q” or “quit”.
#
# Finally, if a sentence is entered that contains a word that is not in
# the vocabulary, we handle this gracefully by printing an error message
# and prompting the user to enter another sentence.
#

def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    #if Algorithm is not 'CNN':
      #input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)  
    # Use appropriate device
    input_batch = input_batch.to(device)
    #lengths = lengths.to(device)
    #### changed to...
    lengths = lengths.to('cpu')
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words

      
from nltk.translate.bleu_score import sentence_bleu 

if Dataset is 1:
  if Algorithm is 'RNN':
    Result_file = os.path.join(corpus, "1_RNN_BLEU_Scores.txt") #will be written to below as a csv
  if Algorithm is 'LSTM':
    Result_file = os.path.join(corpus, "1_LSTM_BLEU_Scores.txt") #will be written to below as a csv
  if Algorithm is 'MLP':
    Result_file = os.path.join(corpus, "1_MLP_BLEU_Scores.txt") #will be written to below as a csv
  if Algorithm is 'DNN':
    Result_file = os.path.join(corpus, "1_DNN_BLEU_Scores.txt") #will be written to below as a csv   
if Dataset is 2:        
  if Algorithm is 'RNN':
    Result_file = os.path.join(corpus, "2_RNN_BLEU_Scores.txt") #will be written to below as a csv
  if Algorithm is 'LSTM':
    Result_file = os.path.join(corpus, "2_LSTM_BLEU_Scores.txt") #will be written to below as a csv
  if Algorithm is 'MLP':
    Result_file = os.path.join(corpus, "2_MLP_BLEU_Scores.txt") #will be written to below as a csv
  if Algorithm is 'DNN':
    Result_file = os.path.join(corpus, "2_DNN_BLEU_Scores.txt") #will be written to below as a csv      


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    count = 0
    with open(Result_file, 'w', encoding='utf-8') as outputfile:
      writer = csv.writer(outputfile, delimiter=')', lineterminator='\n')
      while(1):
          try:          
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

            # BLEU Scoring:
            ref = output_words[:]
            cand = input_sentence
            BLEU_score = sentence_bleu(ref, cand)
            print("BLEU Score: ", BLEU_score)
            results = []
            count = count + 1
            results.append(count)                          
            results.append(BLEU_score)
            writer.writerow(results)

          except KeyError:
            print("Error: Encountered unknown word.")


# Run Model

In [75]:
######################################################################
# Run Model
# ---------
#
# Finally, it is time to run our model!
#
# Regardless of whether we want to train or test the chatbot model, we
# must initialize the individual encoder and decoder models. In the
# following block, we set our desired configurations, choose to start from
# scratch or set a checkpoint to load from, and build and initialize the
# models. Feel free to play with different model configurations to
# optimize performance.
#

# Configure models
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'

hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
checkpoint_iter = 4000
#loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                            '{}_checkpoint.tar'.format(checkpoint_iter))


# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models

#### OLD CNN CODE - DOESN'T WORK!!! #######
#if Algorithm is 'CNN':
  #encoder = EncoderCNN(hidden_size, 3,  embedding, encoder_n_layers, dropout) # (self, hidden_size, kernel_size, embedding, n_layers=1, dropout=0) ### set Kernal size to 3 locally for now
  #decoder = DecoderCNN(attn_model, embedding, 3, hidden_size, voc.num_words, decoder_n_layers, dropout)  ### (self, attn_model, embedding, kernel_size, hidden_size, output_size, n_layers=1, dropout=0.1) ### set Kernal size to 3 locally for now
  #encoder = EncoderCNN(hidden_size, 3,  embedding, encoder_n_layers, dropout) # (self, hidden_size, kernel_size, embedding, n_layers=1, dropout=0) ### set Kernal size to 3 locally for now
  #decoder = DecoderCNN(attn_model, embedding, 3, hidden_size, voc.num_words, decoder_n_layers, dropout)  ### (self, attn_model, embedding, kernel_size, hidden_size, output_size, n_layers=1, dropout=0.1) ### set Kernal size to 3 locally for now
  
#else:
encoder = Encoder(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)


if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


# Run Training

In [76]:
######################################################################
# Run Training
# ~~~~~~~~~~~~
#
# Run the following block if you want to train the model.
#
# First we set training parameters, then we initialize our optimizers, and
# finally we call the ``trainIters`` function to run our training
# iterations.
#

# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 5000 ####was 4000 
print_every = 1
save_every = 500

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# If you have cuda, configure cuda to call
for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()
    
# Run training iterations
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...


<IPython.core.display.Javascript object>

Streaming output truncated to the last 5000 lines.
Iteration: 1; Percent complete: 0.0%; Average loss: 9.9396
Iteration: 2; Percent complete: 0.0%; Average loss: 9.9128
Iteration: 3; Percent complete: 0.1%; Average loss: 9.8854
Iteration: 4; Percent complete: 0.1%; Average loss: 9.8553
Iteration: 5; Percent complete: 0.1%; Average loss: 9.7774
Iteration: 6; Percent complete: 0.1%; Average loss: 9.7136
Iteration: 7; Percent complete: 0.1%; Average loss: 9.5858
Iteration: 8; Percent complete: 0.2%; Average loss: 9.2655
Iteration: 9; Percent complete: 0.2%; Average loss: 8.9398
Iteration: 10; Percent complete: 0.2%; Average loss: 8.5177
Iteration: 11; Percent complete: 0.2%; Average loss: 7.7275
Iteration: 12; Percent complete: 0.2%; Average loss: 7.7743
Iteration: 13; Percent complete: 0.3%; Average loss: 7.6681
Iteration: 14; Percent complete: 0.3%; Average loss: 7.6688
Iteration: 15; Percent complete: 0.3%; Average loss: 7.0631
Iteration: 16; Percent complete: 0.3%; Average loss: 7.152

# Run Evaluation

In [79]:

######################################################################
# Run Evaluation
# ~~~~~~~~~~~~~~
#]
# To chat with your model, run the following block.
#

# Set dropout layers to eval mode
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

# Begin chatting (uncomment and run the following line to begin)
evaluateInput(encoder, decoder, searcher, voc)


######################################################################
# Conclusion
# ----------
#
# That’s all for this one, folks. Congratulations, you now know the
# fundamentals to building a generative chatbot model! If you’re
# interested, you can try tailoring the chatbot’s behavior by tweaking the
# model and training parameters and customizing the data that you train
# the model on.
#
# Check out the other tutorials for more cool deep learning applications
# in PyTorch!
#

> When did Beyonce start becoming popular


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Bot: october in the th century in the th century in the th century in the th century from th century from th century
BLEU Score:  0.47897362544357464
> What areas did Beyonce compete in when she was growing up\


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Bot: dallas hanks in the summer he begun playing at his own findings begun playing at his own findings begun playing at his
BLEU Score:  0.18371774488960746
> In what city and state did Beyonce  grow up 
Bot: florence albania albania east florida and north florida the bronze territory of alta california became made walk through became an nuevo destinations
BLEU Score:  0.3075189572962212
> In which decade did Beyonce become famous
Bot: th in the th century in the th century and the th century in the th century had part of the west territory
BLEU Score:  0.3926948355286693
> In what RB group was she the lead singer
Bot: the soul victoria was insufficient into mid victoria into the hinkley jury in mid victoria was still an absolute monarchy victoria victoria
BLEU Score:  0.2479333541921006
> What album made her a worldwide known artist
Bot: a warm often initially often initially often initially to official home often official point must be held
BLEU Score:  0.4818587613668846
> Who manag

KeyboardInterrupt: ignored

# Results

**See Exec Summary for training error vs iteration**

CORNELL TEST PAIRS:

1. ['there .', 'where ?']
2. ['you have my word . as a gentleman', 'you re sweet .']
3. ['hi .', 'looks like things worked out tonight huh ?']
4. ['you know chastity ?', 'i believe we share an art instructor']
5. ['have fun tonight ?', 'tons']
6. ['well no . . .', 'then that s all you had to say .']
7. ['then that s all you had to say .', 'but']
8. ['but', 'you always been this selfish ?']
9. ['do you listen to this crap ?', 'what crap ?']
10. ['what good stuff ?', 'the real you .']


In [78]:
print("CORNELL DATASET")
print('---------------')
print("CORNELL RNN BLEU SCORES:")
lines = open(os.path.join(corpus, "1_RNN_BLEU_Scores.txt"), encoding='utf-8').\
    read().strip().split('\n')
print(lines)    

print("CORNELL LSTM BLEU SCORES:")
lines = open(os.path.join(corpus, "1_LSTM_BLEU_Scores.txt"), encoding='utf-8').\
        read().strip().split('\n')
print(lines)

print("CORNELL MLP BLEU SCORES:")
lines = open(os.path.join(corpus, "1_MLP_BLEU_Scores.txt"), encoding='utf-8').\
        read().strip().split('\n')
print(lines)

print("CORNELL DNN BLEU SCORES:")
lines = open(os.path.join(corpus, "1_DNN_BLEU_Scores.txt"), encoding='utf-8').\
        read().strip().split('\n')
print(lines)

CORNELL DATASET
---------------
CORNELL RNN BLEU SCORES:
['1\t0.6223329772884784', '2\t0.7226568811456053', '3\t1.0', '4\t0.3639412530979476', '5\t0.5773502691896257', '6\t0.4001601601922499', '7\t0.3168634029243711', '8\t0.9036020036098448', '9\t0.28517539529041497', '10\t0.6964705665515708']
CORNELL LSTM BLEU SCORES:
['1\t0.6223329772884784', '2\t0.3976353643835253', '3\t0.8408964152537145', '4\t0.4005274484725572', '5\t0.6389431042462724', '6\t0.4428500142691474', '7\t0.6838911999336902', '8\t0.9036020036098448', '9\t0.7311104457090247', '10\t0.4005274484725572']
CORNELL MLP BLEU SCORES:
['1\t0.7311104457090247', '2\t0.20690996611611379', '3\t0.7071067811865475', '4\t0.16195570128532405', '5\t0.4924790605054523', '6\t0.6262844962765469', '7\t0.32450414850472864', '8\t0.9036020036098448', '9\t0.43472087194499137', '10\t0.17534823156623092']
CORNELL DNN BLEU SCORES:
['1)0.5555238068023582', '2)0.1531305080006172', '3)0.8408964152537145', '4)0.14025775160081472', '5)0.3381307292971254'

SQuAD 2.0 TEST PAIRS:

** Issues with my Colab GPU the day before submission so was not able to capture the BLEU scores for the SQuAD 2.0 Dataset correctly **

1. ['When did Beyonce start becoming popular] [in the late s\n]'
2. ['What areas did Beyonce compete in when she was growing up] [singing and dancing\n]'
3. ['In what city and state did Beyonce  grow up] [Houston Texas\n]'
4. ['In which decade did Beyonce become famous][late s\n]'
5. ['In what RB group was she the lead singer][Destinys Child\n]'
6. ['What album made her a worldwide known artist][Dangerously in Love\n]'
7. ['Who managed the Destinys Child group][Mathew Knowles\n]'
8. ['When did Beyonc rise to fame][late s\n]'
9. ['What role did Beyonc have in Destinys Child][lead singer\n'
10. ['What was the first album Beyonc released as a solo artist][Dangerously in Love\n]'


In [ ]:
print("SQuAD 2.0 DATASET")
print('---------------')
print("SQuAD  RNN BLEU SCORES:")
#lines = open(os.path.join(corpus, "2_RNN_BLEU_Scores.txt"), encoding='utf-8').\
    #read().strip().split('\n')
#print(lines)    

#print("SQuAD  LSTM BLEU SCORES:")
#lines = open(os.path.join(corpus, "2_LSTM_BLEU_Scores.txt"), encoding='utf-8').\
        #read().strip().split('\n')
#print(lines)

#print("SQuAD  MLP BLEU SCORES:")
#lines = open(os.path.join(corpus, "2_MLP_BLEU_Scores.txt"), encoding='utf-8').\
#        read().strip().split('\n')
#print(lines)

#print("SQuAD  DNN BLEU SCORES:")
#lines = open(os.path.join(corpus, "2_DNN_BLEU_Scores.txt"), encoding='utf-8').\
#        read().strip().split('\n')
#print(lines)